In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.set_random_seed(2018)

## 加载数据

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


## 参数

In [4]:
lr = 0.001
training_iters = 100000
batch_size = 128

n_inputs = 28   #MNIST data input
n_steps = 28    #time steps
n_hidden_units = 128  # neurons in hidden layer 隐藏神经元个数
n_classes = 10  #MNIST classes (0-9)

## tf 图输入

In [5]:
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

In [6]:
# 定义权重
weights = {
    # (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (10, )
    'out':tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

biases = {
    # (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
}

In [7]:
def RNN(X, weights, biases):
    X = tf.reshape(X, [-1, n_inputs])
    
    X_in = tf.matmul(X, weights['in']) + biases['in']
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    # cell
    
    # basic LSTM Cell
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    
    # you have 2 options  for following step
    # 1. tf.nn.rnn(cell, inputs)
    # 2. tf.nn.dynamic_rnn(cell, inputs)
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)
    
    # hidden layers for output as the final results
    outputs = tf.unstack(tf.transpose(outputs, [1, 0, 2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']
    
    return results

In [8]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs, 
            y: batch_ys
        })
        if step % 20 ==0:
            print(sess.run(accuracy, feed_dict={
                x: batch_xs,
                y: batch_ys
            }))
        step += 1

0.25
0.5859375
0.8125
0.7734375
0.8046875
0.859375
0.890625
0.890625
0.9296875
0.953125
0.9296875
0.96875
0.9296875
0.8671875
0.9453125
0.9375
0.9765625
0.9765625
0.921875
0.9140625
0.9375
0.984375
0.9375
0.9375
0.9609375
0.984375
0.984375
0.953125
0.9921875
0.96875
0.9453125
0.953125
0.9609375
0.984375
0.9453125
0.9296875
0.9609375
0.984375
0.984375
0.984375
